In [1]:
import requests 
import bs4 
import pandas as pd
import re

## Scrape Each Categories to Get All Meaningful Links

In [2]:
def get_all_links(url, base_link):
    links = []
    try: r = requests.get(url.split('?')[0])
    except: 
        print("wrong url:", url)
        return []
    soup = bs4.BeautifulSoup(r.text, "html.parser").body
    if not soup: return []
    
    contentPTags = soup.findAll('a')
    for paragraphNum, pTag in enumerate(contentPTags):
        relurl = pTag.get('href')
        new_link = re.sub(' ', '%20', urllib.parse.urljoin(base_link, relurl))
        if base_link in new_link:
            links.append(new_link.split('#')[0]) 
    return list(set(links))

In [8]:
def scrape_field(field, base_url = "https://www.heritage.org/"):
    i = 1
    collection = []
    while(True):
        if i % 20 == 0: print(f'   finished, {i} pages')
        url = f'https://www.heritage.org/{field}?page={i}'
        try: r = requests.get(url.split('?')[0])
        except: break
        links = get_all_links(url, base_url)
        count = set([i.split('?')[0] for i in links if f'/{field}' in i])
        if len(count - set(collection)) == 0: break
        collection.extend(links)
        i += 1
    return list(set([i.split('?')[0] for i in collection]))

In [6]:
base_url = "https://www.heritage.org/"
r = requests.get(base_url.split('?')[0])
soup = bs4.BeautifulSoup(r.text, 'html.parser')
fields = soup.body.findAll("div", { "class" : "views-field views-field-name" })
fields = [i.text for i in fields]

In [7]:
print(fields)

['Agriculture', 'Education', 'Government Regulation', 'Housing', 'American Founders', 'Conservatism', 'Progressivism', 'Public Opinion', 'Asia', 'Europe', 'Global Politics', 'Middle East', 'Budget and Spending', 'Debt', 'Social Security', 'Coal, Oil, Natural Gas', 'Energy Economics', 'Nuclear Energy', 'Renewable Energy', 'Courts', 'Crime and Justice', 'Election Integrity', 'The Constitution', 'Cybersecurity', 'Space Policy', 'Technology', 'Transportation', 'Defense', 'Homeland Security', 'Immigration', 'Terrorism', 'Gender', 'Life', 'Marriage and Family', 'Religious Liberty', 'Health Care Reform', 'Medicaid', 'Medicare', 'Public Health', 'Hunger and Food Programs', 'Poverty and Inequality', 'Welfare', 'International Economies', 'Markets and Finance', 'Taxes', 'Trade']


In [9]:
sum_collection = []
for f in fields:
    field = "-".join(re.split(', | ', f.lower()))
    print('start:', field)
    add = scrape_field(field)
    print(f'   found {len(add)} links')
    sum_collection.extend(add)
    print(f'   total is {len(sum_collection)} links')

start: agriculture
   finished, 20 pages
   found 302 links
   total is 302 links
start: education
   finished, 20 pages
   finished, 40 pages
   finished, 60 pages
   finished, 80 pages
   finished, 100 pages
   finished, 120 pages
   finished, 140 pages
   found 1230 links
   total is 1532 links
start: government-regulation
   finished, 20 pages
   finished, 40 pages
   finished, 60 pages
   finished, 80 pages
   finished, 100 pages
   finished, 120 pages
   found 1120 links
   total is 2652 links
start: housing
   finished, 20 pages
   found 235 links
   total is 2887 links
start: american-founders
   found 136 links
   total is 3023 links
start: conservatism
   finished, 20 pages
   finished, 40 pages
   found 454 links
   total is 3477 links
start: progressivism
   found 185 links
   total is 3662 links
start: public-opinion
   found 73 links
   total is 3735 links
start: asia
   finished, 20 pages
   found 179 links
   total is 3914 links
start: europe
   found 133 links
   total

In [11]:
links = list(set(sum_collection))
linkDF = pd.DataFrame({'Links': links})
linkDF.head()

,Links
0,https://www.heritage.org/energy-economics/comm...
1,https://www.heritage.org/housing/commentary/no...
2,https://www.heritage.org/budget-and-spending/c...
3,https://www.heritage.org/trade/report/trade-ad...
4,https://www.heritage.org/education/report/prom...


In [12]:
linkDF.to_csv('links.csv', index=False)

## Get Information from Each Url

In [118]:
def get_name(sp):
    types = ['div', 'p', 'a']
    cls = ["expert-card__expert-name","contributors-list__contributor-name", 
           "author-card__name nolink", "author-card__name", "author-card__multi-name", 
           "author-card__mulit-author-info"]
    authors = []
    for c in cls:
        for i in types:
            acs = sp.body.findAll(i, { "class" : c})
            if len(acs) != 0: 
                authors.extend(acs)
                
    cln = [i.text.strip('\n').strip(' ').strip('\n') for i in authors if len(i)!=0]
    return cln

In [211]:
def getTextFromPage(url):
    r = requests.get(url.split('?')[0])
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    
    authors = list(set(get_name(soup)))
    
    date = " ".join(soup.body.findAll("div", { "class" : 
                                "article-general-info" })[0].text.split(' ')[:3])
    
    lst = [i.text.strip('\n') for i in soup.body.findAll("p", {"class" : None})]
    lst += [i.text.strip('\n') for i in soup.body.findAll("p", {"class" : "MsoNormal"})]
    lst = [i.replace('\n', ' ') for i in lst if len(i)>1]
    extra = [i for i in soup.body.findAll("em")]
    if (len(lst)>11) and ('Economy' == lst[11]): lst = lst[12:]
    return authors, date, '\n'.join(lst), extra

In [212]:
a = [i for i in links if '/event/' not in i]
a = [i for i in a if '/staff/' not in i]
a = [i for i in a if '/impact/' not in i]
a = [i for i in a if '/heritage-explains/' not in i]
a = [i for i in a if len(i.split('/'))>4]
a_links = a

In [ ]:
#urls, authors, dates, texts, extra, wrong_urls, index_urls = [], [], [], [], [], [], []
for i, url in enumerate(a_links):
    if i < len(urls): continue
    if i % 50 == 0: print(i)
    try: at, dt, txt, ext = getTextFromPage(url)
    except IndexError: 
        print('index error', i, url)
        index_urls.append(url)
        continue
    if (len(at) == 0) or (len(dt) == 0) or (len(txt)==0): 
        print('length is zero', i, url)
        wrong_urls.append(url)
    urls.append(url)
    authors.append(at)
    dates.append(dt)
    texts.append(txt)
    extra.append(ext)

In [219]:
scrapedDF = pd.DataFrame({'urls':urls, 'author':authors, 'date':dates, 'text':texts})
scrapedDF['category'] = scrapedDF['urls'].apply(lambda x: x.split('/')[3])

In [231]:
scrapedDF.head()

,urls,author,date,text,category
0,https://www.heritage.org/energy-economics/comm...,[Katie Tubb],"May 14th, 2014",Yesterday the Senate decided to take up legisl...,energy-economics
1,https://www.heritage.org/housing/commentary/no...,"[Norbert J. Michel, Ph.D.]","Sep 13th, 2018",The 2008 financial crisis was a major missed o...,housing
2,https://www.heritage.org/budget-and-spending/c...,[Stephen Moore],"Oct 19th, 2015","It hasn't gotten much attention, but two big b...",budget-and-spending
3,https://www.heritage.org/trade/report/trade-ad...,"[James Sherk, David Muhlhausen]","May 16, 2011",Authors: David Muhlhausen ...,trade
4,https://www.heritage.org/education/report/prom...,[Kirk Johnson],"April 6, 2005","On April 5, 2005, the Georgetown Public Policy...",education


In [230]:
scrapedDF.to_csv('heritage_total_text.csv', index = False)

## Refine dataset

In [74]:
scrapedDF = pd.read_csv('heritage_total_text.csv')

In [137]:
def get_name_em(url):
    r = requests.get(url.split('?')[0])
    sp = bs4.BeautifulSoup(r.text, 'html.parser')
    x = ["".join(i.text.lower().split(' ')) for i in sp.body.findAll("em")]
    target = []
    for i in sp.body.findAll("a", { "class" :None}):
        try: add = i['href']
        except KeyError: continue
        if '/staff/' in add:
            add = add.split('/staff/')[1].split('.')[0]
            if add in x:
                ind = x.index(add)
                target.append(sp.body.findAll("em")[ind].text)
    target.extend(list(set([i.text.split(',')[0] for i in sp.body.findAll('em')])))
    if len(set(target)) == 0: 
        add = set([i.text.strip('\n') for i in soup.body.findAll('i') if len(i.text.strip('\n'))>0])
        target
    return list(set(target))

In [138]:
index = scrapedDF['author'].apply(lambda x: len(x) == 2)
authors = scrapedDF[index]['urls'].apply(lambda x:get_name_em(x))
index = list(scrapedDF[index].index)
for i, ind in enumerate(index):
    scrapedDF.loc[ind]['author'] = authors[i]

In [139]:
index = scrapedDF['author'].apply(lambda x: len(x) == 0)
authors = scrapedDF[index]['urls'].apply(lambda x:get_name_em(x))

In [144]:
authors = list(authors)
index = list(scrapedDF[index].index)
for i, ind in enumerate(index):
    scrapedDF.loc[ind]['author'] = authors[i]

In [147]:
index = scrapedDF['author'].apply(lambda x: len(x) == 0)
scrapedDF[index]

,urls,author,date,text,category
30,https://www.heritage.org/trade/report/myth-and...,[],"February 1, 1981",NaN,trade
208,https://www.heritage.org/education/commentary/...,[],"Jul 22nd, 2003","Here in Washington, where Congress has offered...",education
299,https://www.heritage.org/health-care-reform/co...,[],"Jun 30th, 2003",The Senate is poised to add a prescription dru...,health-care-reform
653,https://www.heritage.org/testimony/the-state-p...,[],\n,Statement before the Subcommittee on Oversight...,testimony
800,https://www.heritage.org/technology/commentary...,[],"Jan 6th, 2006",The debate about anti-terrorism eavesdropping ...,technology
...,...,...,...,...,...
15494,https://www.heritage.org/political-process/rep...,[],"May 25, 2011",So important is the power to borrow money that...,political-process
15607,https://www.heritage.org/social-security/comme...,[],"Aug 2nd, 2005",August marks the fifth anniversary of the Hyde...,social-security
15917,https://www.heritage.org/taxes/report/time-con...,[],"June 4, 1998",NaN,taxes
15946,https://www.heritage.org/welfare/report/obama-...,[],"September 16, 2009",Executive Summary\nSince the beginning of the ...,welfare


In [148]:
base_url = scrapedDF.iloc[15917]['urls']
print(base_url)
r = requests.get(base_url.split('?')[0])
soup = bs4.BeautifulSoup(r.text, 'html.parser')

https://www.heritage.org/taxes/report/time-congress-end-highest-tax-burden-history


In [149]:
soup.body

<body class="path-node page-node-type-report">
<!-- Google Tag Manager (noscript) -->
<noscript><iframe height="0" src="https://www.googletagmanager.com/ns.html?id=GTM-M69S38" style="display:none;visibility:hidden" width="0"></iframe></noscript>
<!-- End Google Tag Manager (noscript) -->
<a class="visually-hidden focusable skip-link" href="#main-content">
      Skip to main content
    </a>
<div class="dialog-off-canvas-main-canvas" data-off-canvas-main-canvas="">
<div class="page__wrapper" id="page-wrapper">
<div id="page">
<div class="navigation-global__fix-wrapper">
<div aria-hidden="true" class="navigation__article-bar">
<a aria-hidden="true" class="heritage-icon-heritage_bell" href="/" title="Heritage Foundation logo"><span class="visually-hidden">Heritage Foundation</span></a>
<a class="navigation__article-title truncate" href="#" role="button" title="back to top"></a>
<div class="navigation__article-progress-bar article-progress"><span class="visually-hidden">Back to Top</span><

In [121]:
get_name_em(scrapedDF.iloc[138]['urls'])

[]

In [129]:
set([i.text.split(',')[0] for i in soup.body.findAll('em')])

{'Mr. Franc'}

In [162]:
#soup.body.findAll("p", { "class" : "Body" })
set([i.text.strip('\n') for i in soup.body.findAll('i') if len(i.text.strip('\n'))>0])

{'Copied', 'Dr. Ronald\nD. Utt'}